In [10]:
import joblib
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc, \
  ConfusionMatrixDisplay, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score, matthews_corrcoef
import warnings
warnings.filterwarnings('ignore')

In [11]:
data = pd.read_csv(r'..\10_fold_cross_validation\train_10folds.csv')
data

,MaxPartialCharge,FpDensityMorgan2,BCUT2D_CHGLO,BCUT2D_MRHI,PEOE_VSA12,PEOE_VSA6,SMR_VSA3,SlogP_VSA3,SlogP_VSA8,EState_VSA6,NumHAcceptors,NumSaturatedCarbocycles,fr_bicyclic,TARGET,Kfold
0,0.335201,1.714286,-2.072068,5.975550,0.000000,12.132734,9.551078,0.000000,38.974594,43.638476,6,0,1,1.0,0
1,0.226791,2.000000,-2.362642,7.150190,5.948339,0.000000,9.967957,0.000000,10.440599,11.336786,6,0,0,1.0,1
2,0.211302,1.772727,-2.089721,7.912349,9.837253,29.297126,4.983979,9.837253,10.902925,24.265468,4,0,1,0.0,1
3,0.226898,1.869565,-2.357502,6.433493,5.948339,35.334614,15.284746,0.000000,11.126903,12.263211,5,0,0,0.0,0
4,0.158370,1.870968,-2.421374,7.991366,0.000000,23.362825,19.935914,6.420822,0.000000,18.460054,9,0,1,1.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,0.152613,2.000000,-2.415835,6.432407,0.000000,35.334614,9.967957,11.343745,11.257379,6.066367,6,0,0,1.0,3
2735,0.335203,1.700000,-2.001320,6.164147,0.000000,30.331835,4.983979,0.000000,10.902925,36.528679,2,0,1,0.0,6
2736,0.264178,1.833333,-2.414068,5.737009,5.948339,38.112943,9.551078,11.784535,11.126903,11.614772,7,0,0,1.0,8
2737,0.338995,1.815789,-2.065301,7.218862,5.907180,59.014740,24.544948,18.386966,43.634305,6.196844,9,0,2,0.0,5


In [12]:
def run(fold, data):
    # load the full training data with folds
    df = data
    # all columns are features except target and kfold columns
    features = [
        f for f in df.columns if f not in ("TARGET", "Kfold")
    ]
    # get training data using folds
    df_train = df[df.Kfold != fold].reset_index(drop=True)
    # get validation data using folds
    df_valid = df[df.Kfold == fold].reset_index(drop=True)
    # get training data
    X_train = df_train[features].values
    # get validation data
    X_valid = df_valid[features].values
    # initialize Logistic Regression model
    model = GaussianNB()
    model.fit(X_train, df_train.TARGET.values)
    valid_preds = model.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(df_valid.TARGET.values, valid_preds)
    y_pred = model.predict(X_valid)
    y_true = df_valid.TARGET.values
    accuracy = accuracy_score(y_true,y_pred)
    precision_1 = precision_score(y_true,y_pred,pos_label=1)
    precision_0 = precision_score(y_true,y_pred,pos_label=0)
    recall_1 = recall_score(y_true,y_pred,pos_label=1)
    recall_0 = recall_score(y_true,y_pred,pos_label=0)
    f1score = f1_score(y_true,y_pred)
    kappa = cohen_kappa_score(y_true,y_pred)
    MCC = matthews_corrcoef(y_true,y_pred)
    print(f"Fold = {fold}, AUC = {auc}, Accuracy = {accuracy}, \
          Precision_1 = {precision_1}, Precision_0 = {precision_0}\
          Recall_1 = {recall_1}, Recall_0 = {recall_0}, F1Score = {f1score}, kappa = {kappa}, MCC = {MCC}")
    
    return auc, accuracy, precision_1, precision_0, recall_1, recall_0, f1score, kappa, MCC, model

In [13]:
aucs, accuracies, precisions_1, precisions_0, recalls_1, recalls_0, f1scores, kappas, MCCs = [], [], [], [], [], [], [], [], []

for fold_ in range(10):
    auc, accuracy, precision_1, precision_0, recall_1, recall_0, f1score, kappa, MCC, model = run(fold_, data)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions_1.append(precision_1)
    precisions_0.append(precision_0)
    recalls_1.append(recall_1)
    recalls_0.append(recall_0)
    f1scores.append(f1score)
    kappas.append(kappa)
    MCCs.append(MCC)
    #filename = 'SVM_' + str(fold_) + '.pkl'
    #joblib.dump(model, filename)
print("\n")
print(f"Mean Scores: AUC = {np.mean(np.array(aucs))}, \
      Accuracy = {np.mean(np.array(accuracies))}, \
      Precision_1 = {np.mean(np.array(precisions_1))}, Precision_0 = {np.mean(np.array(precisions_0))}\
      Recall_1 = {np.mean(np.array(recalls_1))}, Recall_0 = {np.mean(np.array(recalls_0))}\
      F1Score = {np.mean(np.array(f1scores))} \
      Kappa = {np.mean(np.array(kappas))} \
      MCC = {np.mean(np.array(MCCs))}")

Fold = 0, AUC = 0.8823460888811231, Accuracy = 0.8211678832116789,           Precision_1 = 0.7068965517241379, Precision_0 = 0.9050632911392406          Recall_1 = 0.845360824742268, Recall_0 = 0.807909604519774, F1Score = 0.7699530516431924, kappa = 0.6255786714261811, MCC = 0.6322778418057137
Fold = 1, AUC = 0.8811811986720252, Accuracy = 0.8503649635036497,           Precision_1 = 0.7592592592592593, Precision_0 = 0.9096385542168675          Recall_1 = 0.845360824742268, Recall_0 = 0.8531073446327684, F1Score = 0.8, kappa = 0.6810153898574594, MCC = 0.6835231022267169
Fold = 2, AUC = 0.8366823926844894, Accuracy = 0.7627737226277372,           Precision_1 = 0.6212121212121212, Precision_0 = 0.8943661971830986          Recall_1 = 0.845360824742268, Recall_0 = 0.7175141242937854, F1Score = 0.7161572052401747, kappa = 0.5204372879530401, MCC = 0.5387078240482529
Fold = 3, AUC = 0.859630729803716, Accuracy = 0.8138686131386861,           Precision_1 = 0.6982758620689655, Precision_0 = 0

In [14]:
fold_metrics = pd.DataFrame(columns=['Accuracy','AUC','Precision_1','Precision_0','Recall_1','Recall_0','F1score','Kappa','MCC'])
fold_metrics['Accuracy'] = np.array(accuracies)
fold_metrics['AUC'] = np.array(aucs)
fold_metrics['Precision_1'] = np.array(precisions_1)
fold_metrics['Precision_0'] = np.array(precisions_0)
fold_metrics['Recall_1'] = np.array(recalls_1)
fold_metrics['Recall_0'] = np.array(recalls_0)
fold_metrics['F1score'] = np.array(f1scores)
fold_metrics['Kappa'] = np.array(kappas)
fold_metrics['MCC'] = np.array(MCCs)
fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
0,0.821168,0.882346,0.706897,0.905063,0.845361,0.807910,0.769953,0.625579,0.632278
1,0.850365,0.881181,0.759259,0.909639,0.845361,0.853107,0.800000,0.681015,0.683523
2,0.762774,0.836682,0.621212,0.894366,0.845361,0.717514,0.716157,0.520437,0.538708
3,0.813869,0.859631,0.698276,0.898734,0.835052,0.802260,0.760563,0.610296,0.616832
4,0.791971,0.873551,0.658730,0.905405,0.855670,0.757062,0.744395,0.573954,0.587932
5,0.802920,0.892830,0.666667,0.924138,0.886598,0.757062,0.761062,0.599003,0.616666
6,0.824818,0.891549,0.702479,0.921569,0.876289,0.796610,0.779817,0.637266,0.648013
7,0.748175,0.856311,0.601449,0.897059,0.855670,0.689266,0.706383,0.497421,0.521205
8,0.846715,0.893258,0.728814,0.935897,0.895833,0.820225,0.803738,0.680156,0.689907
9,0.772894,0.864289,0.637097,0.885906,0.822917,0.745763,0.718182,0.533102,0.545363


In [15]:
fold_metrics.loc[10,:] = [np.mean(np.array(accuracies)), np.mean(np.array(aucs)), np.mean(np.array(precisions_1)),
                               np.mean(np.array(precisions_0)), np.mean(np.array(recalls_1)), np.mean(np.array(recalls_0)),
                            np.mean(np.array(f1scores)), np.mean(np.array(kappas)), np.mean(np.array(MCCs))]

fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
0,0.821168,0.882346,0.706897,0.905063,0.845361,0.807910,0.769953,0.625579,0.632278
1,0.850365,0.881181,0.759259,0.909639,0.845361,0.853107,0.800000,0.681015,0.683523
2,0.762774,0.836682,0.621212,0.894366,0.845361,0.717514,0.716157,0.520437,0.538708
3,0.813869,0.859631,0.698276,0.898734,0.835052,0.802260,0.760563,0.610296,0.616832
4,0.791971,0.873551,0.658730,0.905405,0.855670,0.757062,0.744395,0.573954,0.587932
5,0.802920,0.892830,0.666667,0.924138,0.886598,0.757062,0.761062,0.599003,0.616666
6,0.824818,0.891549,0.702479,0.921569,0.876289,0.796610,0.779817,0.637266,0.648013
7,0.748175,0.856311,0.601449,0.897059,0.855670,0.689266,0.706383,0.497421,0.521205
8,0.846715,0.893258,0.728814,0.935897,0.895833,0.820225,0.803738,0.680156,0.689907
9,0.772894,0.864289,0.637097,0.885906,0.822917,0.745763,0.718182,0.533102,0.545363


In [16]:
fold_metrics.loc[11,:] = [np.std(np.array(accuracies)), np.std(np.array(aucs)), np.std(np.array(precisions_1)),
                               np.std(np.array(precisions_0)), np.std(np.array(recalls_1)), np.std(np.array(recalls_0)),
                            np.std(np.array(f1scores)), np.std(np.array(kappas)), np.std(np.array(MCCs))]

fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
0,0.821168,0.882346,0.706897,0.905063,0.845361,0.807910,0.769953,0.625579,0.632278
1,0.850365,0.881181,0.759259,0.909639,0.845361,0.853107,0.800000,0.681015,0.683523
2,0.762774,0.836682,0.621212,0.894366,0.845361,0.717514,0.716157,0.520437,0.538708
3,0.813869,0.859631,0.698276,0.898734,0.835052,0.802260,0.760563,0.610296,0.616832
4,0.791971,0.873551,0.658730,0.905405,0.855670,0.757062,0.744395,0.573954,0.587932
5,0.802920,0.892830,0.666667,0.924138,0.886598,0.757062,0.761062,0.599003,0.616666
6,0.824818,0.891549,0.702479,0.921569,0.876289,0.796610,0.779817,0.637266,0.648013
7,0.748175,0.856311,0.601449,0.897059,0.855670,0.689266,0.706383,0.497421,0.521205
8,0.846715,0.893258,0.728814,0.935897,0.895833,0.820225,0.803738,0.680156,0.689907
9,0.772894,0.864289,0.637097,0.885906,0.822917,0.745763,0.718182,0.533102,0.545363


In [17]:
fold_metrics.index = ['Fold_0','Fold_1','Fold_2','Fold_3','Fold_4','Fold_5','Fold_6','Fold_7','Fold_8','Fold_9','Mean','Std']
fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
Fold_0,0.821168,0.882346,0.706897,0.905063,0.845361,0.807910,0.769953,0.625579,0.632278
Fold_1,0.850365,0.881181,0.759259,0.909639,0.845361,0.853107,0.800000,0.681015,0.683523
Fold_2,0.762774,0.836682,0.621212,0.894366,0.845361,0.717514,0.716157,0.520437,0.538708
Fold_3,0.813869,0.859631,0.698276,0.898734,0.835052,0.802260,0.760563,0.610296,0.616832
Fold_4,0.791971,0.873551,0.658730,0.905405,0.855670,0.757062,0.744395,0.573954,0.587932
Fold_5,0.802920,0.892830,0.666667,0.924138,0.886598,0.757062,0.761062,0.599003,0.616666
Fold_6,0.824818,0.891549,0.702479,0.921569,0.876289,0.796610,0.779817,0.637266,0.648013
Fold_7,0.748175,0.856311,0.601449,0.897059,0.855670,0.689266,0.706383,0.497421,0.521205
Fold_8,0.846715,0.893258,0.728814,0.935897,0.895833,0.820225,0.803738,0.680156,0.689907
Fold_9,0.772894,0.864289,0.637097,0.885906,0.822917,0.745763,0.718182,0.533102,0.545363


# Bayesian Optimization

In [18]:
from hyperopt import hp
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [19]:
class BayesianOPT:
    def __init__(self, data, space):
        self.space = space
        self.data = data
        self.trials = Trials()
    
    def _split_data(self, fold):
        df = self.data
        # all columns are features except target and kfold columns
        features = [
            f for f in df.columns if f not in ("TARGET", "Kfold")
        ]
        # get training data using folds
        df_train = df[df.Kfold != fold].reset_index(drop=True)
        # get validation data using folds
        df_valid = df[df.Kfold == fold].reset_index(drop=True)
        # get training data
        X_train = df_train[features].values
        # get validation data
        X_test = df_valid[features].values
        y_train = df_train.TARGET.values
        y_test = df_valid.TARGET.values
        
        return X_train, y_train, X_test, y_test
    
    def _objective(self,space):
        clf= GaussianNB(var_smoothing=space["var_smoothing"])
        
        
        accuracies = []
        for fold_ in range(10):
            X_train, y_train, X_test, y_test = self._split_data(fold_)
            evaluation = [( X_train, y_train), ( X_test, y_test)]
        

            clf.fit(X_train, y_train)


            pred = clf.predict(X_test)
            accuracy = accuracy_score(y_test, pred>0.5)
            accuracies.append(accuracy)
        
        final_accuracy = np.mean(np.array(accuracies))
        #print ("SCORE:", final_accuracy)
        return {'loss': -final_accuracy, 'status': STATUS_OK }
    
    def search_hyperparameters(self):

        best_hyperparams = fmin(fn = self._objective,
                                space = self.space,
                                algo = tpe.suggest,
                                max_evals = 500,
                                trials = self.trials)
        return best_hyperparams

In [20]:
space = {'var_smoothing' : hp.uniform('var_smoothing',1e-10,1e-1)} 

In [21]:
bayes_opt = BayesianOPT(data, space)
best_params = bayes_opt.search_hyperparameters()
print(best_params)

100%|█████████████████████████████████████████████| 500/500 [00:48<00:00, 10.27trial/s, best loss: -0.7962648057538568]
{'var_smoothing': 1.0021713716172379e-05}


In [14]:
print(best_params)

{'colsample_bytree': 0.9611638174070645, 'gamma': 0.5344801495975495, 'learning_rate': 0.048882041230470526, 'max_depth': 14.0, 'min_child_weight': 0.0, 'reg_alpha': 0.0, 'reg_lambda': 1.164764580743543}


In [15]:
bayes_opt.trials.trials

[{'state': 2,
  'tid': 0,
  'spec': None,
  'result': {'loss': -0.8857263174781422, 'status': 'ok'},
  'misc': {'tid': 0,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'colsample_bytree': [0],
    'gamma': [0],
    'learning_rate': [0],
    'max_depth': [0],
    'min_child_weight': [0],
    'reg_alpha': [0],
    'reg_lambda': [0]},
   'vals': {'colsample_bytree': [0.9152009182780575],
    'gamma': [4.3979445312182275],
    'learning_rate': [0.0572141448813827],
    'max_depth': [7.0],
    'min_child_weight': [1.0],
    'reg_alpha': [26.0],
    'reg_lambda': [0.23651710383646707]}},
  'exp_key': None,
  'owner': None,
  'version': 0,
  'book_time': datetime.datetime(2023, 3, 26, 12, 47, 24, 632000),
  'refresh_time': datetime.datetime(2023, 3, 26, 12, 47, 27, 167000)},
 {'state': 2,
  'tid': 1,
  'spec': None,
  'result': {'loss': -0.8842664634635439, 'status': 'ok'},
  'misc': {'tid': 1,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workd

# Train the model with optimized hyperparameters

In [16]:
def run(fold, data, best_hyper_parameters):
    # load the full training data with folds
    df = data
    # all columns are features except target and kfold columns
    features = [
        f for f in df.columns if f not in ("TARGET", "Kfold")
    ]
    # get training data using folds
    df_train = df[df.Kfold != fold].reset_index(drop=True)
    # get validation data using folds
    df_valid = df[df.Kfold == fold].reset_index(drop=True)
    # get training data
    X_train = df_train[features].values
    # get validation data
    X_valid = df_valid[features].values
    # initialize Logistic Regression model
    model = XGBClassifier(n_jobs = -1,
                          colsample_bytree= best_hyper_parameters["colsample_bytree"], 
                          gamma= best_hyper_parameters["gamma"], 
                          max_depth= int(best_hyper_parameters["max_depth"]), 
                          min_child_weight= best_hyper_parameters["min_child_weight"], 
                          reg_alpha= best_hyper_parameters["reg_alpha"], 
                          reg_lambda= best_hyper_parameters["reg_lambda"])
    
    model.fit(X_train, df_train.TARGET.values)
    valid_preds = model.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(df_valid.TARGET.values, valid_preds)
    y_pred = model.predict(X_valid)
    y_true = df_valid.TARGET.values
    accuracy = accuracy_score(y_true,y_pred)
    precision_1 = precision_score(y_true,y_pred,pos_label=1)
    precision_0 = precision_score(y_true,y_pred,pos_label=0)
    recall_1 = recall_score(y_true,y_pred,pos_label=1)
    recall_0 = recall_score(y_true,y_pred,pos_label=0)
    f1score = f1_score(y_true,y_pred)
    kappa = cohen_kappa_score(y_true,y_pred)
    MCC = matthews_corrcoef(y_true,y_pred)
    print(f"Fold = {fold}, AUC = {auc}, Accuracy = {accuracy}, \
          Precision_1 = {precision_1}, Precision_0 = {precision_0}\
          Recall_1 = {recall_1}, Recall_0 = {recall_0}, F1Score = {f1score}, kappa = {kappa}, MCC = {MCC}")
    
    return auc, accuracy, precision_1, precision_0, recall_1, recall_0, f1score, kappa, MCC, model

In [17]:
aucs, accuracies, precisions_1, precisions_0, recalls_1, recalls_0, f1scores, kappas, MCCs = [], [], [], [], [], [], [], [], []

for fold_ in range(10):
    auc, accuracy, precision_1, precision_0, recall_1, recall_0, f1score, kappa, MCC, model = run(fold_, data, best_params)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions_1.append(precision_1)
    precisions_0.append(precision_0)
    recalls_1.append(recall_1)
    recalls_0.append(recall_0)
    f1scores.append(f1score)
    kappas.append(kappa)
    MCCs.append(MCC)
    filename = 'XGBoost_' + str(fold_) + '.pkl'
    joblib.dump(model, filename)
print("\n")
print(f"Mean Scores: AUC = {np.mean(np.array(aucs))}, \
      Accuracy = {np.mean(np.array(accuracies))}, \
      Precision_1 = {np.mean(np.array(precisions_1))}, Precision_0 = {np.mean(np.array(precisions_0))}\
      Recall_1 = {np.mean(np.array(recalls_1))}, Recall_0 = {np.mean(np.array(recalls_0))}\
      F1Score = {np.mean(np.array(f1scores))} \
      Kappa = {np.mean(np.array(kappas))} \
      MCC = {np.mean(np.array(MCCs))}")

Fold = 0, AUC = 0.9680820082707206, Accuracy = 0.9598540145985401,           Precision_1 = 0.9479166666666666, Precision_0 = 0.9662921348314607          Recall_1 = 0.9381443298969072, Recall_0 = 0.9717514124293786, F1Score = 0.9430051813471502, kappa = 0.9120205499445385, MCC = 0.9120497223728334
Fold = 1, AUC = 0.9729163026384763, Accuracy = 0.9343065693430657,           Precision_1 = 0.9438202247191011, Precision_0 = 0.9297297297297298          Recall_1 = 0.865979381443299, Recall_0 = 0.9717514124293786, F1Score = 0.9032258064516129, kappa = 0.8536411656478129, MCC = 0.8554529191182063
Fold = 2, AUC = 0.9726250800862019, Accuracy = 0.9233576642335767,           Precision_1 = 0.8958333333333334, Precision_0 = 0.9382022471910112          Recall_1 = 0.8865979381443299, Recall_0 = 0.943502824858757, F1Score = 0.8911917098445595, kappa = 0.8320392317123008, MCC = 0.8320658458108835
Fold = 3, AUC = 0.9630147358611452, Accuracy = 0.927007299270073,           Precision_1 = 0.8888888888888888